In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.7 MB/s eta 0:00:00


In [ ]:
pip install dask[dataframe]


INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

# 설정 값
NFOLDS = 3
SEED = 0
NROWS = None

# 데이터 로딩
data = pd.read_csv('/content/drive/MyDrive/application_train.csv/application_train.csv')
test = pd.read_csv('/content/drive/MyDrive/application_test.csv/application_test.csv')
prev = pd.read_csv('/content/drive/MyDrive/previous_application.csv/previous_application.csv')

# 범주형 데이터를 숫자로 변환
categorical_feats = [f for f in data.columns if data[f].dtype == 'object']
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

gc.enable()  # 가비지 컬렉션 활성화

# 타겟 변수 분리
y_train = data['TARGET']
del data['TARGET']

# 이전 신청 데이터의 범주형 변수 변환
prev_cat_features = [f_ for f_ in prev.columns if prev[f_].dtype == 'object']
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

# 평균과 신청 건수 계산
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

# 데이터 병합 및 결측치 처리
x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train = x_train.fillna(0)  # 결측치 0으로 채우기
x_test = x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

# 제외할 특징 정의
excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

# 교차 검증 설정
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

# 모델 래퍼 클래스 정의
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]

class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]

class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]

class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

# Out-of-Fold 예측 생성 함수
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

# 각 모델의 파라미터 설정
et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter'
}

lightgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.1,
    'num_leaves': 123,
    'colsample_bytree': 0.8,
    'subsample': 0.9,
    'max_depth': 15,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'min_split_gain': 0.01,
    'min_child_weight': 2
}

# 각 모델의 인스턴스 생성
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf=CatBoostClassifier, seed=SEED, params=catboost_params)
lg = LightGBMWrapper(clf=LGBMClassifier, seed=SEED, params=lightgbm_params)

# Out-of-Fold 예측 생성
xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

# 교차 검증 점수 출력
print("XG-CV: {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("ET-CV: {}".format(sqrt(mean_squared_error(y_train, et_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print("CB-CV: {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))

# 스태킹을 위한 새로운 특징 생성
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

# 최종 메타 모델 훈련 및 예측
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train)

# 최종 예측 결과 저장
test['TARGET'] = logistic_regression.predict_proba(x_test)[:, 1]
test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')


0:	total: 117ms	remaining: 23.4s
1:	total: 182ms	remaining: 18.1s
2:	total: 235ms	remaining: 15.4s
3:	total: 294ms	remaining: 14.4s
4:	total: 403ms	remaining: 15.7s
5:	total: 459ms	remaining: 14.9s
6:	total: 516ms	remaining: 14.2s
7:	total: 569ms	remaining: 13.6s
8:	total: 632ms	remaining: 13.4s
9:	total: 703ms	remaining: 13.4s
10:	total: 765ms	remaining: 13.1s
11:	total: 825ms	remaining: 12.9s
12:	total: 892ms	remaining: 12.8s
13:	total: 952ms	remaining: 12.7s
14:	total: 1.02s	remaining: 12.5s
15:	total: 1.07s	remaining: 12.3s
16:	total: 1.13s	remaining: 12.2s
17:	total: 1.21s	remaining: 12.2s
18:	total: 1.27s	remaining: 12.1s
19:	total: 1.35s	remaining: 12.2s
20:	total: 1.45s	remaining: 12.3s
21:	total: 1.54s	remaining: 12.5s
22:	total: 1.65s	remaining: 12.7s
23:	total: 1.73s	remaining: 12.7s
24:	total: 1.86s	remaining: 13s
25:	total: 1.98s	remaining: 13.2s
26:	total: 2.1s	remaining: 13.4s
27:	total: 2.21s	remaining: 13.6s
28:	total: 2.32s	remaining: 13.7s
29:	total: 2.44s	remaining: